Part two: Apply different models to the data
======================================================

Unimodal vs. Multimodal models in image + text 
--------------------------------------------

This notebook apply the following models to the meme dataset:

| PK                    |Baseline          | Model Key      | Pretrained Key                                   | Baseline Config                     | Custom Config                              |
|-----------------------|------------------|----------------|--------------------------------------------------|-------------------------------------|--------------------------------------------| 
| unimodal_image_grid   | Image-Grid       | unimodal_image | unimodal_image.hateful_memes.images              | configs/unimodal/image.yaml         | configs/unimodal/image_custom.yaml         |
| unimodal_image_region | Image-Region     | unimodal_image | unimodal_image.hateful_memes.features            | configs/unimodal/with_features.yaml | configs/unimodal/with_features_custom.yaml |
| unimodal_text         | Text BERT        | unimodal_text  | unimodal_text.hateful_memes.bert                 | configs/unimodal/bert.yaml          | configs/unimodal/bert_custom.yaml          |
| late_fusion           | Late Fusion      | late_fusion    | late_fusion.hateful_memes                        | configs/late_fusion/defaults.yaml   | configs/late_fusion/defaults_custom.yaml   |
| concat_bert           | ConcatBERT       | concat_bert    | concat_bert.hateful_memes                        | configs/concat_bert/defaults.yaml   | configs/concat_bert/defaults_custom.yaml   |
| mmbt_grid	            | MMBT-Grid        | mmbt           | mmbt.hateful_memes.images                        | configs/mmbt/defaults.yaml          | configs/mmbt/defaults_custom.yaml          |
| mmbt_region           | MMBT-Region      | mmbt           | mmbt.hateful_memes.features                      | configs/mmbt/with_features.yaml     | configs/mmbt/with_features_custom.yaml     |
| vilbert_direct        | ViLBERT          | vilbert        | vilbert.finetuned.hateful_memes.direct           | configs/vilbert/defaults.yaml       | configs/vilbert/defaults_custom.yaml       |
| visual_bert_direct    | Visual BERT      | visual_bert    | visual_bert.finetuned.hateful_memes.direct       | configs/visual_bert/direct.yaml     | configs/visual_bert/direct_custom.yaml     |
| vilbert_from_cc       | ViLBERT CC       | vilbert        | vilbert.finetuned.hateful_memes.from_cc_original | configs/vilbert/from_cc.yaml        | configs/vilbert/from_cc_custom.yaml        |
| visual_bert_from_coco | Visual BERT COCO | visual_bert    | visual_bert.finetuned.hateful_memes.from_coco    | configs/visual_bert/from_coco.yaml  | configs/visual_bert/from_coco_custom.yaml  |

**Parameters configuration**
The parameters executed for each model are in the .yaml configuration files in the configs/ directory.

**Logs output**
The output logs are located in the following directories:
-  Log configuration: The model training and validation logs are in the /logs directory
-  Tensor log configuration: In the /tensor_log directory. This is the input for the tensorboard visualization metrics.

## Import libraries and set PATH information

In [9]:
import sys
import os
import pandas as pd
import numpy as np

PATH_CURRENT = '/home/jupyter/meme_hateful_detection'
PATH_MODEL = f'{PATH_CURRENT}/models'
PATH_DATA = f'{PATH_CURRENT}/data/raw'
PATH_MEMES_DATASET = f'{PATH_DATA}/datasets/hateful_memes/defaults/annotations'
PATH_SAVE = f'{PATH_CURRENT}/save'
PATH_LOGS = f'{PATH_CURRENT}/logs'
PATH_TENSOR = f'{PATH_CURRENT}/tensor_logs'
PATH_REPO = f'{PATH_SAVE}/reports'
PATH_DATA_OUT = f'{PATH_CURRENT}/data/processed'

os.environ
os.environ['MMF_DATA_DIR'] = PATH_DATA
os.environ['MMF_SAVE_DIR'] = PATH_SAVE
os.environ['MMF_LOG_DIR']  = PATH_LOGS
os.environ['MMF_REPORT_DIR']  = PATH_REPO
os.environ['MMF_TENSORBOARD_LOGDIR']  = PATH_TENSOR
os.environ['MMF_USER_DIR']  = PATH_CURRENT
os.environ['OC_DISABLE_DOT_ACCESS_WARNING'] = '1'
# print(os.environ)

if PATH_CURRENT not in sys.path:
    sys.path.append(PATH_CURRENT)
if PATH_MODEL not in sys.path:
    sys.path.append(PATH_MODEL)
if PATH_DATA not in sys.path:
    sys.path.append(PATH_DATA)
# print(sys.path)

import warnings
warnings.filterwarnings('ignore')

## Define basic functions

In [16]:
def unify_models():
    '''
    This function unify the datasets of all the models executed'''
    df_test = pd.read_json(f'{PATH_MEMES_DATASET}/dev.jsonl', lines=True, orient='records')
    df_unimodal_image_grid = pd.read_csv(f'{PATH_REPO}/unimodal_image_grid_val.csv', sep = ',')
    df_unimodal_image_region = pd.read_csv(f'{PATH_REPO}/unimodal_image_region_val.csv', sep = ',')
    df_unimodal_text = pd.read_csv(f'{PATH_REPO}/unimodal_text_val.csv', sep = ',')
    df_late_fusion = pd.read_csv(f'{PATH_REPO}/late_fusion_val.csv', sep = ',')
    df_concat_bert = pd.read_csv(f'{PATH_REPO}/concat_bert_val.csv', sep = ',')
    df_mmbt_grid = pd.read_csv(f'{PATH_REPO}/mmbt_grid_val.csv', sep = ',')
    df_mmbt_region = pd.read_csv(f'{PATH_REPO}/mmbt_region_val.csv', sep = ',')
    df_vilbert_direct = pd.read_csv(f'{PATH_REPO}/vilbert_direct_val.csv', sep = ',')
    df_visual_bert_direct = pd.read_csv(f'{PATH_REPO}/visual_bert_direct_val.csv', sep = ',')
    df_vilbert_from_cc = pd.read_csv(f'{PATH_REPO}/vilbert_from_cc_val.csv', sep = ',')
    df_visual_bert_from_coco = pd.read_csv(f'{PATH_REPO}/visual_bert_from_coco_val.csv', sep = ',')

    df_all = df_unimodal_image_grid.merge(df_unimodal_image_region, on='id', suffixes=('_unimodal_img_grid', '_unimodal_img_region'))
    df_all = df_all.merge(df_unimodal_text, on='id')
    df_all = df_all.merge(df_late_fusion, on='id', suffixes=('_unimodal_text', '_late_fusion'))
    df_all = df_all.merge(df_concat_bert, on='id')
    df_all = df_all.merge(df_mmbt_grid, on='id', suffixes=('_concat_bert', '_mmbt_grid'))
    df_all = df_all.merge(df_mmbt_region, on='id')
    df_all = df_all.merge(df_vilbert_direct, on='id', suffixes=('_mmbt_region', '_vilbert_direct'))
    df_all = df_all.merge(df_visual_bert_direct, on='id')
    df_all = df_all.merge(df_vilbert_from_cc, on='id', suffixes=('_visual_bert_direct', '_vilbert_from_cc'))
    df_all = df_all.merge(df_visual_bert_from_coco, on='id')
    df_all = df_all.merge(df_test, on='id', suffixes=('_visual_bert_from_coco', '_real_class'))
    df_all.head()
    return(df_all)

## 1. Load data configuration info

For simplicity purposes we store all the data information in a csv called model_config located in models/ directory

In [11]:
df_model_config = pd.read_csv(f'{PATH_MODEL}/model_config.csv', sep = ',')
df_model_config.head(11)

,baseline,model_key,pretrained_key,save_dir,baseline_config,custom_config
0,Image-Grid,unimodal_image,unimodal_image.hateful_memes.images,unimodal_image_grid,configs/unimodal/image.yaml,configs/unimodal/image_custom.yaml
1,Image-Region,unimodal_image,unimodal_image.hateful_memes.features,unimodal_image_region,configs/unimodal/with_features.yaml,configs/unimodal/with_features_custom.yaml
2,Text BERT,unimodal_text,unimodal_text.hateful_memes.bert,unimodal_text,configs/unimodal/bert.yaml,configs/unimodal/bert_custom.yaml
3,Late Fusion,late_fusion,late_fusion.hateful_memes,late_fusion,configs/late_fusion/defaults.yaml,configs/late_fusion/defaults_custom.yaml
4,ConcatBERT,concat_bert,concat_bert.hateful_memes,concat_bert,configs/concat_bert/defaults.yaml,configs/concat_bert/defaults_custom.yaml
5,MMBT-Grid,mmbt,mmbt.hateful_memes.images,mmbt_grid,configs/mmbt/defaults.yaml,configs/mmbt/defaults_custom.yaml
6,MMBT-Region,mmbt,mmbt.hateful_memes.features,mmbt_region,configs/mmbt/with_features.yaml,configs/mmbt/with_features_custom.yaml
7,ViLBERT,vilbert,vilbert.finetuned.hateful_memes.direct,vilbert_direct,configs/vilbert/defaults.yaml,configs/vilbert/defaults_custom.yaml
8,Visual BERT,visual_bert,visual_bert.finetuned.hateful_memes.direct,visual_bert_direct,configs/visual_bert/direct.yaml,configs/visual_bert/direct_custom.yaml
9,ViLBERT CC,vilbert,vilbert.finetuned.hateful_memes.from_cc_original,vilbert_from_cc,configs/vilbert/from_cc.yaml,configs/vilbert/from_cc_custom.yaml


## 2. Apply the model to the data

We have two ways of doing it:

### 2.1. Automatically execute all the models
These can be a very demanding task, each model takes aprox. 10 hours and requires one GPU at least

In [ ]:
#Training model 
for index, row in df_model_config.iterrows():
    model_baseline = row['baseline']
    model_key = row['model_key']
    model_pretrained_key = row['pretrained_key']
    baseline_config = row['baseline_config']
    custom_config = row['custom_config']
    save_dir = row['save_dir']
    str_pret_eval = f'!MMF_SAVE_DIR="{PATH_SAVE}/{save_dir}" mmf_run config={custom_config} model={model_key} dataset=hateful_memes run_type=train'
    print(str_pret_eval)
    !{str_pret_eval}

### 2.2. Execute each model independently

In [ ]:
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/unimodal_image_grid" mmf_run config=configs/unimodal/image_custom.yaml model=unimodal_image dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/unimodal_image_region" mmf_run config=configs/unimodal/with_features_custom.yaml model=unimodal_image dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/unimodal_text" mmf_run config=configs/unimodal/bert_custom.yaml model=unimodal_text dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/late_fusion" mmf_run config=configs/late_fusion/defaults_custom.yaml model=late_fusion dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/concat_bert" mmf_run config=configs/concat_bert/defaults_custom.yaml model=concat_bert dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/mmbt_region" mmf_run config=configs/mmbt/with_features_custom.yaml model=mmbt dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/mmbt_grid" mmf_run config=configs/mmbt/defaults_custom.yaml model=mmbt dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/visual_bert_direct" mmf_run config=configs/visual_bert/direct_custom.yaml model=visual_bert dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/vilbert_direct" mmf_run config=configs/vilbert/defaults_custom.yaml model=vilbert dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/vilbert_from_cc" mmf_run config=configs/vilbert/from_cc_custom.yaml model=vilbert dataset=hateful_memes run_type=train
!MMF_SAVE_DIR="/home/jupyter/meme_hateful_detection/save/visual_bert_from_coco" mmf_run config=configs/visual_bert/from_coco_custom.yaml model=visual_bert dataset=hateful_memes run_type=train

## 3. Test our model

Just as the modeling part we can execute automatically the testing of our models:

In [ ]:
for index, row in df_model_config.iterrows():
    model_baseline = row['baseline']
    model_key = row['model_key']
    model_pretrained_key = row['pretrained_key']
    baseline_config = row['baseline_config']
    custom_config = row['custom_config']
    str_pred_test = f'MMF_SAVE_DIR="{PATH_SAVE}/{save_dir}" mmf_predict config={custom_config} model={model_key} dataset=hateful_memes run_type=test'
    print(str_pred_test)
    !{str_pred_test}    

## 4. Evaluate our model

Finally we validate all the models in the next cell

In [3]:
for index, row in df_model_config.iterrows():
    model_baseline = row['baseline']
    model_key = row['model_key']
    model_pretrained_key = row['pretrained_key']
    baseline_config = row['baseline_config']
    custom_config = row['custom_config']
    save_dir = row['save_dir']
    str_pred_eval = f'mmf_predict config={custom_config} model={model_key} dataset=hateful_memes run_type=val checkpoint.resume_zoo={model_pretrained_key}'
    print(str_pred_eval)
    !{str_pred_eval}

mmf_predict config=configs/unimodal/image_custom.yaml model=unimodal_image dataset=hateful_memes run_type=val checkpoint.resume_zoo=unimodal_image.hateful_memes.images
mmf_predict config=configs/unimodal/with_features_custom.yaml model=unimodal_image dataset=hateful_memes run_type=val checkpoint.resume_zoo=unimodal_image.hateful_memes.features
mmf_predict config=configs/unimodal/bert_custom.yaml model=unimodal_text dataset=hateful_memes run_type=val checkpoint.resume_zoo=unimodal_text.hateful_memes.bert
mmf_predict config=configs/late_fusion/defaults_custom.yaml model=late_fusion dataset=hateful_memes run_type=val checkpoint.resume_zoo=late_fusion.hateful_memes
mmf_predict config=configs/concat_bert/defaults_custom.yaml model=concat_bert dataset=hateful_memes run_type=val checkpoint.resume_zoo=concat_bert.hateful_memes
mmf_predict config=configs/mmbt/defaults_custom.yaml model=mmbt dataset=hateful_memes run_type=val checkpoint.resume_zoo=mmbt.hateful_memes.images
mmf_predict config=con

## 5. Unify results in a single dataset

For this purpose we'll use the outputs of the validation process. To accomplish this we've changed the mmf library so the file output has the format "[save_dir]_var.csv". Finally we store all unified information in a pickle file

In [17]:
df_models_output = unify_models()
df_models_output.to_pickle(f'{PATH_DATA_OUT}/models_output.pkl')

In [18]:
df_models_output.head()

,id,proba_unimodal_img_grid,label_unimodal_img_grid,proba_unimodal_img_region,label_unimodal_img_region,proba_unimodal_text,label_unimodal_text,proba_late_fusion,label_late_fusion,proba_concat_bert,...,label_vilbert_direct,proba_visual_bert_direct,label_visual_bert_direct,proba_vilbert_from_cc,label_vilbert_from_cc,proba,label_visual_bert_from_coco,img,label_real_class,text
0,8291,0.547192,0,0.510450,1,0.532066,1,0.529459,1,0.542653,...,0,0.995959,0,0.513247,1,0.585236,0,img/08291.png,1,white people is this a shooting range
1,46971,0.518059,0,0.508191,1,0.502754,1,0.508565,1,0.516585,...,0,0.996868,0,0.748288,0,0.650853,1,img/46971.png,1,bravery at its finest
2,3745,0.519284,0,0.509476,1,0.510536,1,0.503237,1,0.520288,...,0,0.930861,0,0.604765,1,0.721242,1,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,0.537087,0,0.506875,1,0.509638,1,0.506610,1,0.507785,...,0,0.996248,0,0.713064,0,0.588802,0,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,0.530560,0,0.508561,1,0.511030,1,0.513430,1,0.509205,...,1,0.913100,0,0.884496,1,0.709484,0,img/80243.png,1,mississippi wind chime


## Optional: Train, test and evaluate one model

In [ ]:
# Loading model in your code
model_baseline = 'Image-Grid'
model_key = 'unimodal_image'
model_pretrained_key = 'unimodal_image.hateful_memes.images'
baseline_config = 'configs/unimodal/image.yaml'
custom_config = 'configs/unimodal/image_custom.yaml'

# Training
str_run_train = f'mmf_run config={custom_config} model={model_key} dataset=hateful_memes'
# Test
str_run_eval  = f'mmf_run config={custom_config} model={model_key} dataset=hateful_memes run_type=test'
# Predictions val
str_pred_eval = f'mmf_predict config={custom_config} model={model_key} dataset=hateful_memes run_type=val'